# Homework 5

## Part 1: Playlist Continuation

In the first part of this homework, you'll build a simple music playlist continuation system based on collaborative filtering and audio features using a dataset derived from the [Million Playlist Dataset (MPD)](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#challenge-dataset). 

Later in this part, you will test each of your systems based on these four playlists:

- Playlist 1:
    - Britney Spears "...Baby One More Time"
    - Kelly Clarkson "Since U Been Gone"
- Playlist 2:
    - Radiohead "Exit Music",
    - Muse "Citizen Erased"
- Playlist 3:
    - Dr. Dre "Xxplosive"
    - Eric B. & Rakim "Paid In Full"
- Playlist 4:
    - Rage Against The Machine "Bombtrack"
    - Audioslave "Like A Stone"

In [ ]:
import numpy as np
import os
import json
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import glob

### Unzip embeddings.zip
This compressed file contains a list of audio embeddings, extracted from MusiCNN (Pons and Serra, 2019)

Each file is named as `[tid].npy` format.

For example, `TRCSBUG128F422318C.npy` is an audio embedding of "The Navy Song" by Billy Talent.

In [ ]:
#!unzip embeddings.zip

## 1. Construct samples using the training set (1 mark)

**Inputs**
- playlists: a dictionary where each key is a playlist index and the corresponding value is a list of track dictionaries. Each track dictionary contains metadata such as `tid`, `artist_name`, and `track_name`.

**Outputs**
- data: a list of interaction tuples. Each tuple contains three elements:
    - the playlist index (i.e., user index) (e.g., 0, 1, 2...),
    - the track index (mapped from tid) (e.g., 0, 1, 2...),
    - a binary label indicating interaction (1 for positive, 0 for negative). For example, the tuple `(0, 0, 1)` means that playlist 0 included track 0, while `(1, 2, 0)` means playlist 1 did not include track 2. The number of positive samples (label = `1`) must be equal to the number of negative samples (label = 0). Since there are more negative samples than positive ones, you should randomly select negative samples.
- tid_to_idx: a dictionary mapping each unique tid to a unique index (integer). (e.g., `{'TRPYFRU128F427777C':0 ...}`)
- idx_to_tid: the inverse of `tid_to_idx`. A dictionary mapping each unique index back to its corresponding tid. (e.g., `{0: 'TRPYFRU128F427777C' ...}`)
- tid_to_meta: a dictionary mapping each tid to its metadata as a tuple: `(artist_name, track_name)`.
    For example: `tid_to_meta['TRXIOLV12903CF1882'] = ('The Antlers', 'Wake')`.

**One thing to note**
- When mapping TIDs to indexes, the order of TIDs should not matter. For example, `{‘TRPYFRU128F427777C': 0, …}` and `{‘TRXIOLV12903CF1882': 0, …}` are both valid `tid_to_idx` mappings. However, the dictionaries `tid_to_idx`,` idx_to_tid`, and `tid_to_meta` must include all TIDs present in the playlists, and the indices should range from `0` to `len(playlists) - 1`.

In [ ]:
def build_interaction_samples(playlists):
    # Your code here
    return data, tid_to_idx, idx_to_tid, tid_to_meta

## 2. Implement a PyTorch dataset and dataloader and train the provided WRMF model (this part is given)

Implement `WRMFDataset`, a PyTorch dataset that takes a data index as input and returns the corresponding playlist index, track index, and binary interaction label.
Then, optimize the provided WRMF model for 10 epochs. Use the model’s `compute_loss` method when calculating the training loss (1 mark).

**Inputs**
- `data`: A list of interaction tuples created with `build_interaction_samples`

**Outputs**
- `playlist_index`: An index corresponding to the playlist
- `track_index`: An index corresponding to the TID
- `label`: A binary label (0 or 1) indicating whether the playlist includes the track.

In [ ]:
class WRMFDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return item[0], item[1], item[2]

In [ ]:
class WRMF(nn.Module):
    def __init__(self, num_users, num_items, num_factors, alpha=40.0, lambda_reg=0.1):
        super(WRMF, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)
        self.alpha = alpha
        self.lambda_reg = lambda_reg
        nn.init.normal_(self.user_factors.weight, std=0.01)
        nn.init.normal_(self.item_factors.weight, std=0.01)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

    def compute_loss(self, user, item, feedback):
        prediction = torch.sigmoid(self.forward(user, item))
        confidence = 1 + self.alpha * feedback
        loss = (confidence * (1 - feedback - prediction)**2).sum()
        loss += self.lambda_reg * (
            self.user_factors(user).pow(2).sum() +
            self.item_factors(item).pow(2).sum()
        )
        return loss / user.shape[0]

## 3. Recommend using the trained WRMF model (1 mark)

#### `get_playlist_embedding()`

**Inputs**
- `model`: A trained WRMF model
- `playlist`: A list of tids representing a playlist
- `tid_to_idx`: As implemented above

**Output**
- `playlist_embedding`: An average tensor of embeddings, where each embedding corresponds to a track in the playlist (Use `item_factors` of the model.)

#### `generate_recommendations()`

**Inputs**
- `model`: A trained WRMF model
- `playlist`: A list of tids representing a playlist
- `all_item_embeddings`: A tensor containing all track embeddings, retrieved from the trained WRMF model (Use the weights of `item_factors` of the model.)
- `idx_to_tid`: As implemented above
- `tid_to_idx`: As implemented above
- `N`: Number of recommendations to return

**Details**
- Convert the playlist's tids into embedding indices using `tid_to_idx`
- Compute the average playlist embedding using `get_playlist_embedding`
- Calculate cosine similarity between the playlist embedding and `all_item_embeddings`
- Identify the most similar items

**Outputs**
- `similarities`: Similarity scores for the 10 most similar items that are not already in the playlist
- `recommendations`: A list of track IDs (e.g. "TRCNVLA12903CF6052", not indices) for those items

In [ ]:
def get_playlist_embedding(model, playlist, tid_to_idx):
    # Your code here
    return item_embeddings.mean(dim=0)

def generate_recommendations(model, playlist, all_item_embeddings, idx_to_tid, tid_to_idx, N):
    # Your code here
    # (line below is a hint, feel free to delete)
    return [similarities[int(idx)].item() for idx in recommended_indices if idx not in playlist_item_ids][:N],\
           [idx_to_tid[int(idx)] for idx in recommended_indices if idx not in playlist_item_ids][:N]

# 4. Generate rankings on the test set using the trained WRMF model (2 marks)

#### `make_cf_rankings()`

**Inputs**
- `test_playlists`: A dictionary of playlists loaded from `test_playlists.json`
- `all_item_embeddings`: A tensor containing all track embeddings, retrieved from the trained WRMF model

**Outputs**
- `rankings`: A dictionary where each key is a playlist index, and the corresponding value is a list of track IDs sorted in descending order of similarity to the embedding of the first two tracks in the playlist
- `targets`: A dictionary where each key is a playlist index, and the corresponding value is a list of track IDs indices from the third track onward (i.e., tracks that should be recommended)

**Details**
- Note that the recommendations should be generated based on **the first two tracks** in the playlist and the targets are the third track onward.
- You could probably return either ids (0,1,2) or track IDs ("TRCNVLA12903CF6052" etc.), as long as you are consistent between both outputs.

**Instructions**
- Initialize `rankings` and `targets` as empty dictionaries.
- For each playlist:
    - Filter out any TIDs that do not exist in the training set.
    - Obtain the average embedding of the first two tracks in the playlist using `get_playlist_embedding`.
    - Calculate the cosine similarity between the playlist embedding and `all_item_embeddings`.
    - Sort the results in descending order and exclude tracks already present in the first two tracks of the playlist.
    - Store the ranking in the `rankings` dictionary, where the key is the playlist index and the value is the ranking.
    - Store the tracks from the third one onward in the `targets` dictionary, where the key is the playlist index and the value is the list of tracks

**Outputs**
- Return the constructed `rankings` and `targets` dictionaries.

In [ ]:
def make_cf_rankings(model, test_playlists, all_item_embeddings, idx_to_tid, tid_to_idx):
    # Your code here
    return rankings, targets

## 5. Evaluate using MRR (1 mark)

### `get_mrr()`

**Inputs**

- `rankings`: A dictionary mapping each playlist index to a list of recommended item indices, sorted by predicted relevance.
- `targets`: A dictionary mapping each playlist index to a list of ground-truth item indices (i.e., tracks that should have been recommended).

**Output**

- `mrr`: A float value representing the **Mean Reciprocal Rank (MRR)** across all playlists.

**Details**

- MRR measures how early in the recommendation list the correct items appear. For each target item that appears in the ranking, we compute the reciprocal of its rank (1-based). The MRR for a single playlist is the average reciprocal rank of all its targets. The final output is the mean over all playlists.

**Formula:**
Let $R_i$ be the rank of the $i$-th correct (target) item in the recommended list. For a playlist with $n$ correct items:

$$\text{MRR}_{\text{playlist}} = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{R_i}$$

$$\text{MRR} = \frac{1}{|\text{Playlists}|} \sum_{\text{playlist}} \text{MRR}_{\text{playlist}}$$



In [ ]:
def get_mrr(rankings, targets):
    # Your code here
    return np.average(mrrs)

## 6. Evaluate using Precision (1 mark)

### `get_precision()`

**Inputs**

- `rankings`: A dictionary mapping each playlist index to a list of recommended item indices.
- `targets`: A dictionary mapping each playlist index to a list of ground-truth item indices.

**Output**

- `precision`: A float value representing **Precision\@10**, averaged over all playlists.

**Details**
* Precision\@10 evaluates how many of the top 10 recommended items are relevant. For each playlist, it computes the number of recommended items in the top 10 that are also in the ground-truth list, divided by the number of ground-truth items.

**Formula:**

$$\text{Precision@10} = \frac{|\text{Top-10 Recommendations} \cap \text{Targets}|}{|\text{Targets}|}$$

$$\text{Precision} = \frac{1}{|\text{Playlists}|} \sum_{\text{playlist}} \text{Precision@10}_{\text{playlist}}$$

In [ ]:
def get_precision(rankings, targets):
    # Your code here
    return np.average(precisions)

## 7. Recommend Based on Audio Similarity (1 mark)

### `get_average_audio_embeddings()`

**Inputs**
- `playlist`: A list of track IDs (`tid`s) representing a playlist
- `embedding_directory`: A directory path containing `.npy` files, each named after a `tid` and containing audio embeddings.

**Output**
- A single averaged audio embedding vector (NumPy array) representing the playlist

**Details**
- For each track in the playlist, load its audio embedding from the corresponding `.npy` file and compute the mean along the time axis. Then return the average of all such track embeddings as a single playlist-level embedding.


### `get_embeddings()`

**Inputs**
- `embedding_directory`: Path to a directory containing `.npy` files. Each file represents the audio embedding of a track and is named with its corresponding `tid`.

**Outputs**
- `embedding_matrix`: A NumPy array where each row is the mean audio embedding of a track
- `tids`: A list of `tid`s corresponding to each row in `embedding_matrix`

**Details**
- Loads all `.npy` files from the directory
- For each file, computes the average embedding across time (i.e., mean over axis 0)
- Returns the stacked embeddings as a matrix and the list of corresponding `tid`s


### `get_similarity()`

**Inputs**
- `playlist`: A list of `tid`s representing the input playlist
- `playlist_embedding`: A single embedding vector (e.g., from `get_average_audio_embeddings`)
- `embedding_matrix`: A NumPy array of all track embeddings (output from `get_embeddings`)
- `tids`: A list of `tid`s corresponding to the rows of `embedding_matrix`

**Output**
- `rankings`: A list of track indices (mapped from `tid` using `tid_to_idx`), sorted by descending cosine similarity to the input playlist embedding, and excluding tracks already in the input playlist

**Details**
- Computes cosine similarity between the `playlist_embedding` and all track embeddings in `embedding_matrix`
- Sorts the results in descending order of similarity
- Filters out tracks that are already in the playlist
- Return similarities and rankings (similar to `generate_recommendations` above)

In [ ]:
def get_average_audio_embedding(playlist, embedding_directory):
    # Your code here
    return np.mean(embeddings, axis=0)

In [ ]:
def get_embeddings(embedding_directory, tids):
    # Your code here
    return embedding_matrix, tids

def get_similarity(playlist, playlist_embedding, embedding_matrix, tid_to_idx, tids):
    # Your code here
    return sims, rankings

## 8. Generate rankings on the test set using audio similarity (1 mark)

### `make_audio_inference()`

**Inputs**

- `test_playlists`: A dictionary where each key is a playlist index and the corresponding value is a list of track dictionaries. Each track dictionary must contain a `'tid'` field.
- `embedding_matrix`: A NumPy array of all track embeddings (output from `get_embeddings`)
- `tids`: A list of `tid`s corresponding to the rows of `embedding_matrix`

**Outputs**

- `rankings`: A dictionary where each key is a playlist index and the corresponding value is a list of recommended tracks (not present in the input playlist), sorted in descending order of cosine similarity to the audio-based playlist embedding
- `targets`: A dictionary where each key is a playlist index and the corresponding value is a list of ground-truth tracks (i.e., tracks from the third position onward in the playlist)

**Details**

- For each playlist:
  - Use the first two tracks to compute the playlist-level audio embedding using `get_average_audio_embeddings`
  - Generate a ranked list of similar tracks (excluding existing ones) using `get_similarity`
  - Store the ranking in `rankings` and the remaining tracks (from position 3 onward) in `targets`
  - As above, could return either tracks or IDs, as long as you're consistent

In [ ]:
def make_audio_rankings(directory, test_playlists, embedding_matrix, tid_to_idx, tids):
    # Your code here
    return rankings, targets

## Part 2: Sound Synthesis

In the second part of this homework, you'll implement some basic utilities of a sound synthesizer (ADSR and LFOs). Specifically, you'll modify the amplitude of a sound using ADSR and the pitch using an LFO.

## 9. Implement ADSR envelope (1 mark).

- Attack Time: How long it takes for the synth to reach maximum amplitude (i.e., 1).
- Decay Time: How long it takes for the synth to decrease to the sustain level after the attack phase.
- Sustain Level: The amplitude level maintained after the decay phase while the note is held.
- Release Time: How long it takes for the synth to fade out completely after the note is released.

#### `adsr_envelope()`

**Inputs**
* duration: The duration (in seconds) during which the note is held.
* attack_time: A float representing the attack time (in seconds).
* decay_time: A float representing the decay time (in seconds).
* sustain_level: A float representing the sustain level, ranging from 0 to 1.
* release_time: A float representing the release time (in seconds).

**Outputs**
* envelope: A 1D NumPy array of floating-point values between 0 and 1, with a total length of int((duration + release_time) * sr) samples. This array is intended to be multiplied with a waveform (with constant amplitude 1) to apply the ADSR envelope. 

In [ ]:
def adsr_envelope(duration, attack_time, decay_time, sustain_level, release_time, sr=44100):
    # Your code here
    return envelope

### 10. Modulate the cutoff frequency of the low-pass filter using an LFO (1 mark)

- Base Cutoff Frequency: The central or starting cutoff frequency.
- LFO Frequency: How fast the LFO oscillates. For example, A 0.5 Hz LFO periodically modulates the cutoff once every 2 seconds.
- LFO Depth: The amount of intensity by which the LFO affects the cutoff frequency. For example, if the cutoff is 200Hz, the actual cutoff will swing between 800Hz and 1200Hz.

![](./question10.png)

#### `get_lfo()`

**Inputs**

* `base_cutoff`: A float representing the base cutoff frequency in Hz.
* `lfo_depth`: A float representing the depth of modulation in Hz.
* `lfo_freq`: A float representing the frequency of the LFO in Hz.
* `t`: A 1D NumPy array representing evenly spaced time values over the duration of the signal. The number of samples is determined by the sampling rate and the total duration.

**Outputs**

* `lfo`: A 1D NumPy array of the same length as `t`, where each element represents the modulated cutoff frequency at that point in time.


In [ ]:
def get_lfo(base_cutoff, lfo_depth, lfo_freq, t):
    # Your code here
    pass